In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [2]:
import numpy
import tflearn
import tensorflow as tf
import random
import json

C:\Users\dilee\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\dilee\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\dilee\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\dilee\anaconda3\envs\tensorflow\lib\site-packages

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [3]:
import pickle

In [4]:
with open("intents.json") as file:
    data = json.load(file)

In [5]:
#print(data)

In [6]:
#try
words = []
labels = []
docs_x= []
docs_y = []


In [7]:
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
    
    if intent["tag"] not in labels:
        labels.append(intent["tag"])

In [8]:
words = [stemmer.stem(w.lower()) for w in words if w not in "?"]
words = sorted(list(set(words)))

In [9]:
labels = sorted(labels)

In [10]:
#Training and Testing output
training = []
output = []

In [11]:
out_empty = [0 for _ in range(len(labels))]

In [12]:
for x, doc in enumerate(docs_x):
    bag = []
    
    wrds = [stemmer.stem(w) for w in doc]
    
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)
            
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1
    
    training.append(bag)
    output.append(output_row)

In [13]:
training = numpy.array(training)
output = numpy.array(output)

In [14]:
#tf.reset_default_garph() #take care of tf


In [15]:
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [16]:
model = tflearn.DNN(net)

In [17]:
#try:
 #   model.load("model.tflearn")
#except:
model.fit(training, output, n_epoch=2000, batch_size=8, show_metric=True)
model.save("model.tflearn")

Training Step: 21999  | total loss: 0.00449 | time: 0.040s
| Adam | epoch: 2000 | loss: 0.00449 - acc: 1.0000 -- iter: 80/87
Training Step: 22000  | total loss: 0.00425 | time: 0.040s
| Adam | epoch: 2000 | loss: 0.00425 - acc: 1.0000 -- iter: 87/87
--
INFO:tensorflow:C:\Users\dilee\Chatbot file\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [18]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    
    s_words  = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]
    
    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
                
    return numpy.array(bag)

In [19]:
def chat():
    print("Start talking with the bot!(type quit to stop)")
    
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break
            
        results = model.predict([bag_of_words(inp, words)])[0]
        results_index = numpy.argmax(results)
        tag = labels[results_index]
        
        if results[results_index] > 0.75:
            for tg in data['intents']:
                if tg['tag'] == tag:
                    responses = tg['responses']
                
            print(random.choice(responses))
        else:
            print("I did not understand what you are asking, please try again.")
        

In [20]:
chat()

Start talking with the bot!(type quit to stop)
You: Hello
Good to see you again!
You: quit
